In [1]:
import urllib
import pnwkit
import pandas as pd
import sqlalchemy as sq
from sqlalchemy import create_engine
from urllib.parse import quote_plus

In [2]:
#DB Connection
conFormat = urllib.parse.quote_plus(
    'Data Source Name=Franklin;' 
    'Driver={SQL Server};' 
    'Server=FRANKLIN;' 
    'Database=PnWDB;' 
    'Trusted_connection=yes;'
)
#create engine
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(conFormat))

# Extract

In [3]:
#Query GraphQL API using PnwKit-Py Wrapper
#Insert API Key here
kit = pnwkit.QueryKit("5992d973dadb75")

#Query first num of results at once
#Retrieves only fields attributes specified in .query() method
nation_query = kit.query(
    'nations', {'first': 500}, 'id', 'nation_name', 'leader_name', 'num_cities', 'date', 'color', 'last_active',
    'score', 'alliance_id', 'alliance_position', 'gross_domestic_product', 'gross_national_income', 'population', 
    'projects', 'turns_since_last_city', 'turns_since_last_project', 'soldiers', 'tanks', 'aircraft', 'ships', 'missiles', 
    'nukes', 'vacation_mode_turns'
)
#Query alliance field and attributes
alliance_query = kit.query(
    'alliances', {'first': 100}, 'id', 'name', 'color', 'date'
)
#Query treaty field and attributes
treaty_query = kit.query(
    'treaties', {'first': 500}, 'id', 'treaty_type', 'date', 'alliance1_id', 'alliance2_id', 'turns_left'
)
#Query city field and attributes
city_query = kit.query(
    'cities', {'first': 500}, 'id', 'name', 'nation { nation_name }', 'nation_id', 'date', 'infrastructure',
    'land', 'powered'
)

#Paginate queries for all available results in each field
nations = nation_query.paginate("nations")
alliances = alliance_query.paginate("alliances")
treaties = treaty_query.paginate("treaties")
cities = city_query.paginate("cities")

#Initialize empty dictionaries that will later be put converted into Dataframes
#Also set column order here
nation_data = {
    'id': [], 'nation_name': [], 'leader_name': [], 'num_cities': [], 'date': [], 'color': [], 'score': [],
    'alliance_id': [], 'alliance_position': [], 'population': [], 'gross_domestic_product': [], 'gross_national_income': [], 
    'projects': [], 'turns_since_last_city': [], 'turns_since_last_project': [], 'soldiers': [], 'tanks': [], 
    'aircraft': [], 'ships': [], 'missiles': [], 'nukes': [], 'vacation_mode_turns': [], 'last_active': []
}

alliance_data = {
    'id': [], 'name': [], 'color': [], 'date': []
}
treaty_data = {
    'id': [], 'treaty_type': [], 'alliance1_id': [], 'alliance2_id': [], 'date': [], 'turns_left': []
}
city_data = {
    'id': [], 'name': [], 'nation': [], 'nation_id': [], 'date': [], 'infrastructure': [], 'land': []
}

In [5]:
#Convert paginator objects to dictionaries
#Iterate through paginator object: nations
for nation in nations:
    #iterate through empty dict and append values
    for key in nation_data:
        #Special case where key != object atrribute
        if key != 'alliance_position':
            nation_data[str(key)].append(nation.__getattribute__(key))
        else:
            #nested attribute alliance position
            nation_data['alliance_position'].append(nation.alliance_position.name)

#Iterate through paginator object: alliances
for alliance in alliances:
    #iterate through empty dict and append values
    for key in alliance_data:
            alliance_data[str(key)].append(alliance.__getattribute__(key))

#Iterate through paginator object: treaties
for treaty in treaties:
    #iterate through empty dict and append values
    for key in treaty_data:
        treaty_data[str(key)].append(treaty.__getattribute__(key))

#Iterate through paginator object: cities
for city in cities:
    for key in city_data:
        #iterate through empty dict and append values
        if key != 'nation':
            city_data[str(key)].append(city.__getattribute__(key))
        #Special case where key != object atrribute
        else:
            #nested attribute nation_name
            city_data['nation'].append(city.nation.nation_name)

# Transform

In [6]:
#create DF from dictionaries
alliances_df = pd.DataFrame(alliance_data)
#change dtypes to appropriate datetime format
alliances_df['date'] = pd.to_datetime(alliances_df.date).dt.tz_localize(None)
#change column headers to more approprate names
alliances_df.rename(columns={'id': 'alliance_id', 'name': 'alliance_name', 'color': 'color_bloc', 'date': 'founded', }, 
                    inplace=True)
alliances_df.head()

,alliance_id,alliance_name,color_bloc,founded
0,790,Rose,pink,2014-11-04 23:13:16
1,1584,The Knights Radiant,orange,2015-06-29 17:25:50
2,1023,The Fighting Pacifists,white,2015-01-26 15:22:05
3,7452,Cataclysm,white,2020-04-13 19:15:17
4,5875,The Immortals,yellow,2019-08-04 19:30:43


In [7]:
#create DF from dictionaries
nations_df = pd.DataFrame(nation_data)
#change dtypes to appropriate datetime format
nations_df['date'] = pd.to_datetime(nations_df.date).dt.tz_localize(None)
nations_df['last_active'] = pd.to_datetime(nations_df.last_active).dt.tz_localize(None)
#change column headers to more approprate names
nations_df.rename(columns={'id': 'nation_id', 'num_cities': 'cities', 'date': 'founded', 'color': 'color_bloc',
                           'score': 'nation_score', 'gross_domestic_product': 'gdp', 'gross_national_income': 'gni', 
                           'projects':'national_projects', 'vacation_mode_turns':'vm_turns'}, inplace=True)
#select records in nations_df if the alliance id exists in the alliance df
new_nations_df = nations_df[nations_df['alliance_id'].isin(alliances_df['alliance_id'])]
new_nations_df.head()

,nation_id,nation_name,leader_name,cities,founded,color_bloc,nation_score,alliance_id,alliance_position,population,...,turns_since_last_city,turns_since_last_project,soldiers,tanks,aircraft,ships,missiles,nukes,vm_turns,last_active
0,5270,Belleroph,Callisto,40,2014-08-05 00:09:31,pink,8844.74,10523,MEMBER,15837151,...,422,1175,600000,50000,2985,486,0,0,0,2023-05-15 00:33:12
1,582,Nightsilver Woods,Luna,23,2014-08-05 00:09:35,gray,1840.09,615,MEMBER,566,...,8759,8759,0,0,0,0,0,0,0,2022-02-26 21:12:20
2,439,Howell,Malal,32,2014-08-05 00:10:03,beige,3153.33,6088,MEMBER,2596096,...,528,3188,0,0,0,0,0,0,0,2023-05-15 00:11:55
3,300,BEPU,Aenir,46,2014-08-05 00:10:40,green,9655.70,3339,MEMBER,20582927,...,532,3057,228000,36000,2715,435,0,0,0,2023-05-15 04:57:26
4,405,Gladius,Samwise,42,2014-08-05 00:10:41,gray,8597.75,3339,MEMBER,18807719,...,840,454,165000,28750,2310,305,0,0,0,2023-05-13 01:15:47


In [10]:
#create DF from dictionaries
treaties_df = pd.DataFrame(treaty_data)
#change dtypes to appropriate datetime format
treaties_df['date'] = pd.to_datetime(treaties_df.date).dt.tz_localize(None)
#change column headers to more approprate names
treaties_df.rename(columns={'id': 'treaty_id', 'alliance1_id': 'sending_alliance_id', 'alliance2_id': 'receiving_alliance_id',
                            'date': 'date_signed'}, inplace=True)
#select records in treaties_df if the alliance id exists in the alliance df
send_treaties_df = treaties_df[treaties_df['sending_alliance_id'].isin(alliances_df['alliance_id'])]
new_treaties_df = send_treaties_df[send_treaties_df['receiving_alliance_id'].isin(alliances_df['alliance_id'])]
new_treaties_df.head()

,treaty_id,treaty_type,sending_alliance_id,receiving_alliance_id,date_signed,turns_left
0,19893,Protectorate,10455,4271,2022-10-15 13:44:19,980
1,20013,Protectorate,7817,4648,2022-10-31 02:06:21,563
3,20217,NAP,10455,10402,2022-11-17 13:56:16,10
4,20249,NAP,10613,10635,2022-11-21 00:52:13,54
5,20273,MDP,10761,6215,2022-11-24 08:04:09,90


In [12]:
#create DF from dictionaries
cities_df = pd.DataFrame(city_data)
#change dtypes to appropriate datetime format
cities_df['date'] = pd.to_datetime(cities_df.date).dt.tz_localize(None)
#change column headers to more approprate names
cities_df.rename(columns={'id': 'city_id', 'name': 'city_name', 'nation':'nation_name', 'date': 'date_built'}, 
                 inplace=True)
#select records in cities_df if the nation id exists in the nation df
new_cities_df = cities_df[cities_df['nation_id'].isin(new_nations_df['nation_id'])]
new_cities_df.head()

,city_id,city_name,nation_name,nation_id,date_built,infrastructure,land
0,10603,Flanaca,Belleroph,5270,2014-08-05,837.00,4500.0
1,10604,Selemne,Nightsilver Woods,582,2014-08-05,0.14,3020.0
2,10605,Howell,Howell,439,2014-08-05,559.51,5000.0
3,10614,in the city of,BEPU,300,2014-08-05,3000.00,5020.0
4,10615,Green Dragon,Gladius,405,2014-08-05,3000.00,4500.0


# Load into DB

In [19]:
#send tables into DB, preserving column datatypes
alliances_df.to_sql('alliances', con=engine, schema='dbo', if_exists='append', index=False,
                    dtype={'alliance_id': sq.types.SMALLINT(),'alliance_name': sq.types.VARCHAR(), 
                           'founded': sq.types.DATETIME(),'color_bloc': sq.types.VARCHAR()})

-1

In [20]:
#send tables into DB, preserving column datatypes
new_nations_df.to_sql('nations', con=engine, schema='dbo', if_exists='append', index=False,
                  dtype={'nation_id': sq.types.INT(), 'nation_name': sq.types.VARCHAR(100), 'leader_name': sq.types.VARCHAR(100),
                         'cities': sq.types.SMALLINT(), 'founded': sq.types.DATETIME(), 'color_bloc': sq.types.VARCHAR(100),
                         'nation_score': sq.types.FLOAT, 'alliance_id': sq.types.SMALLINT(), 
                         'alliance_position': sq.types.VARCHAR(50), 'population': sq.types.INT(), 'gdp': sq.types.BIGINT, 
                         'gni': sq.types.BIGINT, 'national_projects': sq.types.SMALLINT(), 
                         'turns_since_last_city': sq.types.SMALLINT(), 'turns_since_last_project': sq.types.SMALLINT(), 
                         'soldiers': sq.types.INT(), 'tanks': sq.types.INT(), 'aircraft': sq.types.SMALLINT(), 
                         'ships': sq.types.SMALLINT(), 'vm_turns':sq.types.INT, 'last_active': sq.types.DATETIME()})

-1

In [21]:
#send tables into DB, preserving column datatypes
new_treaties_df.to_sql('treaties', con=engine, schema='dbo', if_exists='append', index=False,
                   dtype={'treaty_id': sq.types.INT(), 'treaty_type': sq.types.VARCHAR(), 'alliance_id': sq.types.SMALLINT(), 
                          'date_signed': sq.types.DATETIME(), 'turns_left': sq.types.SMALLINT()})

new_cities_df.to_sql('cities', con=engine, schema='dbo', if_exists='append', index=False,
                    dtype={'city_id': sq.types.INT, 'city_name': sq.types.VARCHAR(100), 'city_name': sq.types.VARCHAR(100),
                           'nation_name': sq.types.VARCHAR(100), 'nation_id': sq.types.INT, 'date_built': sq.types.Date(),
                           'infrastructure': sq.types.DECIMAL(6,2), 'land': sq.types.DECIMAL(6,2)})

-1